In [40]:
import cv2
import numpy as np

In [52]:
# 입력 동영상 파일 경로
input_file = "/Users/yuna/Homework/IMG_3644.mov"
original = "/Users/yuna/Homework/BrailleBlockOriginal.jpeg"

# 특정 색상 범위 설정 (예: 노란색)
lower_yellow = np.array([10, 20, 200])  # 범위의 하한값 (HSV 형식)
upper_yellow = np.array([30, 80, 255])  # 범위의 상한값 (HSV 형식)

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(input_file)

orig = cv2.imread(original, 0)

# 영상의 너비와 높이 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 동영상의 프레임 레이트 가져오기
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps * 10)  # 10초마다 프레임 추출cv2.fillPoly(traMask, [pts], 255)

In [42]:
def yellow_mask(frame):
    """
    입력된 프레임에 특정 색상 범위에 해당하는 마스크를 적용합니다.
    
    :param frame: 입력 프레임 (BGR 형식)
    :param lower_color: 색상 범위의 하한값 (HSV 형식)
    :param upper_color: 색상 범위의 상한값 (HSV 형식)
    :return: 색상 마스크
    """
    # 입력된 프레임을 HSV 형식으로 변환
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 색상 범위에 해당하는 마스크 생성
    mask = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)
    
    return mask

In [43]:
def histogram_equalization(frame):
    """
    입력된 프레임에 히스토그램 평활화를 적용합니다.
    
    :param frame: 입력 프레임
    :return: 히스토그램 평활화가 적용된 프레임
    """
    # 입력 프레임을 그레이 스케일로 변환
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 히스토그램 평활화 적용
    equalized_frame = cv2.equalizeHist(gray_frame)
    
    return equalized_frame

In [44]:
def otsu_thresholding(mask):
    """
    Otsu의 이진화를 적용하여 이진 마스크를 생성합니다.
    
    :param mask: 입력 마스크
    :return: Otsu의 이진화가 적용된 마스크
    """
    # Otsu의 이진화 적용
    _, otsu_mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_mask

In [45]:
def apply_morphology(mask):
    """
    입력된 마스크에 모폴로지 연산을 적용합니다.
    
    :param mask: 이진 마스크
    :return: 모폴로지 연산이 적용된 마스크
    """
    # 커널 생성
    kernel = np.ones((5, 5), np.uint8)
    
    # motphology 연산 적용 (잡음 제거)
    result = cv2.dilate(mask, kernel, iterations=1)
    result = cv2.dilate(result, kernel, iterations=1)
    result = cv2.dilate(result, kernel, iterations=1)
    #result = cv2.erode(result, kernel, iterations=1)
    
    return result

In [46]:
def process_frame(frame):
    """
    주어진 프레임에 연결 성분 레이블링을 적용합니다.
    
    :param frame: 입력 프레임 (BGR 형식)
    :return: 레이블링된 결과 프레임 (BGR 형식)
    """
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 이진화
    _, binary_image = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 연결 성분 레이블링
    num_labels, labels_im = cv2.connectedComponents(binary_image)
    
    # 레이블링된 결과를 색상 이미지로 변환하여 시각화
    label_hue = np.uint8(179 * labels_im / np.max(labels_im))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # HSV에서 BGR로 변환
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    
    # 배경을 검정으로 변환
    labeled_img[label_hue == 0] = 0
    
    return labeled_img

In [47]:
def braille_detection(frame):
     # 사다리꼴 모양의 마스크 생성
    traMask = np.zeros((height, width), dtype=np.uint8)
    pts = np.array([[0, 1500], [width-100, 1500], [width-300, 500], [300, 600]], dtype=np.int32)
    
    # 마스크 적용
    frame = cv2.bitwise_and(frame, frame, mask=traMask)
    
    # 히스토그램 평활화 적용
    equalized_frame = histogram_equalization(frame)
    
    # 특정 색상 범위에 해당하는 마스크 적용
    color_mask = yellow_mask(frame)
    
    # Otsu의 이진화 적용
    otsu_mask = otsu_thresholding(color_mask)
    
    # 마스크를 적용하여 특정 색상만 남기고 나머지는 가리기
    masked_frame = cv2.bitwise_and(frame, frame, mask=otsu_mask)
    
    # 연결 성분 레이블링 적용
    labeled_frame = process_frame(masked_frame)

    # 모폴로지 연산 적용
    new = apply_morphology(labeled_frame)
    
    return new

In [50]:
def load_video():
    # 동영상 프레임 처리
    count = 0
    while cap.isOpened():
        count += 1
        ret, frame = cap.read()
        if not ret:
            break

        new = braille_detection(frame)
        orig = braille_detection(original)
    
        if count % frame_interval == 0:
            cv2.absdiff(new, orig)
            dif = new - original
            np.array(dif)
            total = np.sum(dif)
            if (total > 300000):
                print("Beep")
    
        # 결과 프레임 표시
        cv2.imshow('Masked Equalized and Labeled Video', new)
    
        # 종료를 위한 키 입력 처리
        if cv2.waitKey(1) & 0xFF == ord('q'):  # 1ms 대기 (더 빠르게 재생)
            break

In [51]:
load_video()

/var/folders/n9/7h5phywx43537thdz_87qp3c0000gn/T/ipykernel_16578/1359856594.py:18: RuntimeWarning: invalid value encountered in divide
  label_hue = np.uint8(179 * labels_im / np.max(labels_im))
/var/folders/n9/7h5phywx43537thdz_87qp3c0000gn/T/ipykernel_16578/1359856594.py:18: RuntimeWarning: invalid value encountered in cast
  label_hue = np.uint8(179 * labels_im / np.max(labels_im))


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'bitwise_and'
> Overload resolution failed:
>  - src1 is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src1'


In [25]:
# 파일 닫기
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1